In [1]:
# Basic Setup
import numpy as np
import pandas as pd


In [2]:
# Read Data
all_data = pd.read_csv('adult.data', sep=',', header=None)
attributes_names = ['Age', 'Workclass','Fnlwgt','Education','Education_Num','Marital_Status',
                    'Occupation','Relationship','Race','Sex','Capital_Gain','Capital_Loss',
                    'Hours_Per_Week','Native_Country','Income']
all_data.columns = attributes_names


Prediction:

In [3]:
def rearrange_data(data, corresponding_attributes):
    '''
        Rearrange all data and use integer to represent each attributes and corresponding labels
        Return:
                1. inputs
                2. corresponding outputs
    '''
    num_data = len(data)
    num_attributes = len(attributes_names) - 1
    class_labels = np.zeros(num_data, dtype='int')
    inputs = np.zeros((num_data, num_attributes), dtype='int')  # Initialize data 

    
    for j in range(num_attributes):    # Rearrange inputs
        dic = {}
        cor_index = 0
        for i in range(num_data):
            if j == 1 or j == 3 or 5 <= j <= 9 or j == 13:
                if data[attributes_names[j]][i] not in dic:
                    dic[data[attributes_names[j]][i]] = cor_index
                    cor_index += 1
                inputs[i][j] = dic[data[attributes_names[j]][i]]
            else:
                inputs[i][j] = data[attributes_names[j]][i]
        if len(dic.keys()) > 0:
            corresponding_attributes[j] = dic
    dic = {}
    cor_index = 0
    for i in range(num_data):         # Rearrange class_labels(output)
        if data['Income'][i] not in dic:
            dic[data['Income'][i]] = cor_index
            cor_index += 1
        class_labels[i] = dic[data['Income'][i]]
    return inputs, class_labels, corresponding_attributes

corresponding_attributes = {}
inputs, class_labels, corresponding_attributes = rearrange_data(all_data, corresponding_attributes)
print(inputs)
print(class_labels)
print(corresponding_attributes)

[[    39      0  77516 ...      0     40      0]
 [    50      1  83311 ...      0     13      0]
 [    38      2 215646 ...      0     40      0]
 ...
 [    58      2 151910 ...      0     40      0]
 [    22      2 201490 ...      0     20      0]
 [    52      6 287927 ...      0     40      0]]
[0 0 0 ... 0 0 1]
{1: {' State-gov': 0, ' Self-emp-not-inc': 1, ' Private': 2, ' Federal-gov': 3, ' Local-gov': 4, ' ?': 5, ' Self-emp-inc': 6, ' Without-pay': 7, ' Never-worked': 8}, 3: {' Bachelors': 0, ' HS-grad': 1, ' 11th': 2, ' Masters': 3, ' 9th': 4, ' Some-college': 5, ' Assoc-acdm': 6, ' Assoc-voc': 7, ' 7th-8th': 8, ' Doctorate': 9, ' Prof-school': 10, ' 5th-6th': 11, ' 10th': 12, ' 1st-4th': 13, ' Preschool': 14, ' 12th': 15}, 5: {' Never-married': 0, ' Married-civ-spouse': 1, ' Divorced': 2, ' Married-spouse-absent': 3, ' Separated': 4, ' Married-AF-spouse': 5, ' Widowed': 6}, 6: {' Adm-clerical': 0, ' Exec-managerial': 1, ' Handlers-cleaners': 2, ' Prof-specialty': 3, ' Other-se